In [ ]:
##Pseudo Code for Analysis
#Read in APIs
#Read CSVs
#All Dates by year

In [ ]:
#Price Per Square foot by Zip Code
    #"Date" "Zip" "Price Per Square foot"
#Schools
    #"Date" "Zip" "School District" "Grade"
#Crime
    #"Date" "Zip" "Type" 
#Income Tax
    #"Date" "Zip" "Income Tax Paid" "Number of returns" Total Charitable Contributions" "Brackets"
#Real Estate Tax
    #"Date" "Zip" "Real Estate taxes paid" "personal property tax"
#Opportunity Zones
    #"Zip"
#Stray Animals
    #"Date" "Zip" "Number of Strays Reported"
#Population
    #"Date" "Zip" "Number of People"
#Construction Permits
    #"Date" "Zip" "Type (Residential vs Commercial)"
#Restaurant, Bars, & Clubs